# 导入模块

In [2]:
import os
os.chdir(r"D:\SPO4Portfolio")
import numpy as np
import pandas as pd
from dateutil.relativedelta import relativedelta
from scipy.optimize import minimize

start_date = pd.to_datetime("2015-01-01")

# 数据准备

In [ ]:
# 所有 ETF
tickers = ["EEM","EFA","JPXN","SPY","XLK",'VTI','AGG','DBC']
n_assets = len(tickers)
equal_weight = np.full(n_assets, 1/n_assets)  # 每个资产的权重都是 1/N

# 读取 log return
return_df = pd.DataFrame()
for ticker in tickers:
    df = pd.read_csv(f"data/FeatureData/{ticker}.csv", parse_dates=["Date"])
    df["Date"] = pd.to_datetime(df["Date"]).dt.normalize()
    df = df.set_index("Date")["log_return"].rename(ticker)
    return_df = pd.concat([return_df, df], axis=1)

# 等权投资

In [8]:
# 所有 ETF
tickers = ["EEM","EFA","JPXN","SPY","XLK",'VTI','AGG','DBC']
n_assets = len(tickers)
equal_weight = np.full(n_assets, 1/n_assets)  # 每个资产的权重都是 1/N

# 读取 log return
return_df = pd.DataFrame()
for ticker in tickers:
    df = pd.read_csv(f"data/FeatureData/{ticker}.csv", parse_dates=["Date"])
    df["Date"] = pd.to_datetime(df["Date"]).dt.normalize()
    df = df.set_index("Date")["log_return"].rename(ticker)
    return_df = pd.concat([return_df, df], axis=1)

# 定义每个月初作为投资起始点

months = 120

results = []

for i in range(months):
    infer_start = start_date + relativedelta(months=i)
    infer_end = (infer_start + relativedelta(months=1)) - pd.Timedelta(days=1)

    try:
        # 获取这个月每天的算术收益率
        arith_return_month = np.expm1(return_df.loc[infer_start:infer_end, tickers].values)
        daily_return = arith_return_month @ equal_weight  # 每日组合收益
        monthly_return = np.prod(1 + daily_return) - 1     # 月组合收益

    except Exception as e:
        print(f"⚠️ 无法计算 {infer_start.strftime('%Y-%m')} 的组合收益：{e}")
        monthly_return = np.nan

    print(f"📅 {infer_start.strftime('%Y-%m')} 等权组合收益：{monthly_return:.4%}")
    results.append((infer_start.strftime('%Y-%m'), monthly_return))

# 保存结果
df_result = pd.DataFrame(results, columns=["Month", "MonthlyReturn"])
df_result["CumulativeReturn"] = (1 + df_result["MonthlyReturn"]).cumprod() - 1
df_result.to_csv("result\8_ticker_1ytrain1yinfer\EqualWeight.csv", index=False)


📅 2015-01 等权组合收益：-2.4495%
📅 2015-02 等权组合收益：5.1161%
📅 2015-03 等权组合收益：-1.6062%
📅 2015-04 等权组合收益：3.0531%
📅 2015-05 等权组合收益：-0.1818%
📅 2015-06 等权组合收益：-1.7653%
📅 2015-07 等权组合收益：-1.2141%
📅 2015-08 等权组合收益：-5.0213%
📅 2015-09 等权组合收益：-2.8099%
📅 2015-10 等权组合收益：6.1463%
📅 2015-11 等权组合收益：-0.9098%
📅 2015-12 等权组合收益：-2.5323%
📅 2016-01 等权组合收益：-4.2087%
📅 2016-02 等权组合收益：-0.9623%
📅 2016-03 等权组合收益：6.4934%
📅 2016-04 等权组合收益：1.0254%
📅 2016-05 等权组合收益：1.0692%
📅 2016-06 等权组合收益：0.8801%
📅 2016-07 等权组合收益：2.7127%
📅 2016-08 等权组合收益：0.5857%
📅 2016-09 等权组合收益：1.6636%
📅 2016-10 等权组合收益：-0.9628%
📅 2016-11 等权组合收益：-0.0484%
📅 2016-12 等权组合收益：1.5982%
📅 2017-01 等权组合收益：2.5089%
📅 2017-02 等权组合收益：2.1485%
📅 2017-03 等权组合收益：0.7866%
📅 2017-04 等权组合收益：0.9256%
📅 2017-05 等权组合收益：1.8678%
📅 2017-06 等权组合收益：0.0509%
📅 2017-07 等权组合收益：2.9213%
📅 2017-08 等权组合收益：0.8788%
📅 2017-09 等权组合收益：1.3422%
📅 2017-10 等权组合收益：3.1722%
📅 2017-11 等权组合收益：1.4006%
📅 2017-12 等权组合收益：1.4853%
📅 2018-01 等权组合收益：4.7345%
📅 2018-02 等权组合收益：-3.1461%
📅 2018-03 等权组合收益：-0.7261%
📅 2018-04 

# 最大化夏普比率

## 夏普比率优化函数

In [5]:
def max_sharpe_portfolio(mu, cov, risk_free=0.0):
    n = len(mu)

    def neg_sharpe(w):
        port_return = np.dot(w, mu)
        port_vol = np.sqrt(np.dot(w, np.dot(cov, w)))
        return -((port_return - risk_free) / port_vol)

    constraints = ({
        'type': 'eq',
        'fun': lambda w: np.sum(w) - 1
    })
    bounds = [(0.0, 1.0)] * n
    w0 = np.array([1.0 / n] * n)

    result = minimize(neg_sharpe, w0, bounds=bounds, constraints=constraints)
    return result.x  # 最优权重


## 月度回测代码

In [9]:
results = []

for i in range(months):
    infer_start = start_date + relativedelta(months=i)
    infer_end = (infer_start + relativedelta(months=1)) - pd.Timedelta(days=1)
    train_start = infer_start - relativedelta(years=1)
    train_end = infer_start - pd.Timedelta(days=1)

    try:
        # 1. 获取训练区间的算术收益率
        train_data = return_df.loc[train_start:train_end, tickers]
        train_return = np.expm1(train_data)
        daily_mu = train_return.mean()
        daily_cov = train_return.cov()

        # 2. 年化收益与协方差
        mu_annual = daily_mu * 252
        cov_annual = daily_cov * 252

        # 3. 求最大夏普比组合
        weights = max_sharpe_portfolio(mu_annual.values, cov_annual.values)

        # 4. 用该组合评估当前月收益
        test_return = np.expm1(return_df.loc[infer_start:infer_end, tickers].values)
        daily_return = test_return @ weights
        monthly_return = np.prod(1 + daily_return) - 1

    except Exception as e:
        print(f"⚠️ {infer_start.strftime('%Y-%m')} 出错：{e}")
        monthly_return = np.nan
        weights = [np.nan] * len(tickers)

    print(f"📅 {infer_start.strftime('%Y-%m')} Max Sharpe 月收益：{monthly_return:.4%}")
    results.append((infer_start.strftime('%Y-%m'), monthly_return))

df_maxsharpe = pd.DataFrame(results, columns=["Month", "MonthlyReturn"])
df_maxsharpe["CumulativeReturn"] = (1 + df_maxsharpe["MonthlyReturn"]).cumprod() - 1
df_maxsharpe.to_csv("result/8_ticker_1ytrain1yinfer/MaxSharpe.csv", index=False)


d:\Anaconda3\envs\DLinear\lib\site-packages\numpy\lib\function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
d:\Anaconda3\envs\DLinear\lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
d:\Anaconda3\envs\DLinear\lib\site-packages\pandas\core\frame.py:10866: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
d:\Anaconda3\envs\DLinear\lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
d:\Anaconda3\envs\DLinear\lib\site-packages\numpy\lib\function_base.py:2748: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


📅 2015-01 Max Sharpe 月收益：-2.4495%
📅 2015-02 Max Sharpe 月收益：-0.2251%
📅 2015-03 Max Sharpe 月收益：0.4718%
📅 2015-04 Max Sharpe 月收益：0.9023%
📅 2015-05 Max Sharpe 月收益：0.5326%
📅 2015-06 Max Sharpe 月收益：-0.9709%
📅 2015-07 Max Sharpe 月收益：0.2169%
📅 2015-08 Max Sharpe 月收益：-3.2971%
📅 2015-09 Max Sharpe 月收益：-2.4097%
📅 2015-10 Max Sharpe 月收益：0.9751%
📅 2015-11 Max Sharpe 月收益：0.0315%
📅 2015-12 Max Sharpe 月收益：-0.8518%
📅 2016-01 Max Sharpe 月收益：-1.1022%
📅 2016-02 Max Sharpe 月收益：-0.6093%
📅 2016-03 Max Sharpe 月收益：0.9376%
📅 2016-04 Max Sharpe 月收益：-0.5442%
📅 2016-05 Max Sharpe 月收益：0.2330%
📅 2016-06 Max Sharpe 月收益：1.6460%
📅 2016-07 Max Sharpe 月收益：1.0497%
📅 2016-08 Max Sharpe 月收益：-0.0920%
📅 2016-09 Max Sharpe 月收益：0.2995%
📅 2016-10 Max Sharpe 月收益：-0.8056%
📅 2016-11 Max Sharpe 月收益：-2.2883%
📅 2016-12 Max Sharpe 月收益：0.7828%
📅 2017-01 Max Sharpe 月收益：0.4913%
📅 2017-02 Max Sharpe 月收益：1.5836%
📅 2017-03 Max Sharpe 月收益：0.0701%
📅 2017-04 Max Sharpe 月收益：0.9216%
📅 2017-05 Max Sharpe 月收益：3.0194%
📅 2017-06 Max Sharpe 月收益：-1.581